# Case Study: Backend

This notebook contains the backend code for the app. We first load the model you've trained (and saved as `"strength_model.pkl"`) in the previous notebook `a` and then make the predictions and optimize the cement's fineness.

The fineness optimization relies on domain knowledge, namely that milling cement 1 micron finer results in a strength increase of 1 MPa.

The backend is built using [FastAPI](https://fastapi.tiangolo.com/), a modern, fast (high-performance), web framework for building APIs with Python. Please note that normally this code would be run as a regular script, not in a notebook.

In [ ]:
import numpy as np
import pandas as pd
import joblib
import nest_asyncio
import uvicorn
from fastapi import Body, FastAPI
from pydantic import BaseModel, Field

In [ ]:
# load your trained model - make sure it's saved under the right name!
model = joblib.load("strength_model.pkl")

In [ ]:
# configure the host and port where the server will run
# host depends on your setup -> "127.0.0.1" if running locally, "0.0.0.0" inside a docker container
host = "127.0.0.1"
port = 8000

In [ ]:
# FastAPI app instance
app = FastAPI()

# pydantic model used for data validation
class CementSample(BaseModel):
    cement: str = Field(description="cement")
    fineness: float = Field(ge=0., le=60., description="fineness")
    material_1: float = Field(ge=-1., le=101., description="material_1")
    material_2: float = Field(ge=-1., le=101., description="material_2")
    material_3: float = Field(ge=-1., le=101., description="material_3")
    material_4: float = Field(ge=-1., le=101., description="material_4")
    material_5: float = Field(ge=-1., le=101., description="material_5")
    material_6: float = Field(ge=-1., le=101., description="material_6")
    material_7: float = Field(ge=-1., le=101., description="material_7")
    material_8: float = Field(ge=-1., le=101., description="material_8")
    material_9: float = Field(ge=-1., le=101., description="material_9")
    material_10: float = Field(ge=-1., le=101., description="material_10")
    material_11: float = Field(ge=-1., le=101., description="material_11")
    material_12: float = Field(ge=-1., le=101., description="material_12")
    material_13: float = Field(ge=-1., le=101., description="material_13")
    material_14: float = Field(ge=-1., le=101., description="material_14")
    material_15: float = Field(ge=-1., le=101., description="material_15")
    target_strength: float = Field(ge=0., le=100., description="Desired compressive strength in MPa")

# endpoint for the base URI, which can be queried with a GET request
@app.get("/")
def home():
    return "Congratulations! Your API is working."

# endpoint for /predict, which can be queried with a POST request to send the cement sample data
@app.post("/predict")
def predict_and_optimize(cement_sample: CementSample):
    target_strength = cement_sample.target_strength
    # transform the given data into a pandas dataframe
    cement_sample = cement_sample.model_dump()
    # you can change the feature names here if your model uses fewer features
    features = ["fineness"] + [f"material_{i}" for i in range(1, 16)]
    x = pd.DataFrame({c: [cement_sample[c]] for c in features}, columns=features)
    # predict strength with the current fineness
    pred_org = model.predict(x)[0]
    # fineness change: 1 micron finer = 1 MPa stronger
    diff = target_strength - pred_org
    fineness_new = cement_sample["fineness"] - diff
    # return the computed values as a JSON
    return {
        "fineness_org": cement_sample["fineness"], 
        "fineness_new": fineness_new, 
        "pred_org": pred_org, 
        "pred_new": target_strength,
    }

By running the following cell you will spin up the server!

This causes the notebook to block (no cells/code can run) until you manually interrupt the kernel. You can do this by clicking on the Kernel tab and then on Interrupt or by entering Jupyter's command mode by pressing the `ESC` key and tapping the `I` key twice.

In [ ]:
# allow the server to be run in this interactive environment
nest_asyncio.apply()
# spin up the server  
uvicorn.run(app, host=host, port=port)